In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

good_tyre_path = '/home/karthik/Desktop/CNN/git/websitecnn/archive/allimages/good'
bad_tyre_path = '/home/karthik/Desktop/CNN/git/websitecnn/archive/allimages/defective'

# Function to load and preprocess images
def load_and_preprocess_images(folder_path, label, target_size=(128, 128)): 
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return np.array(images), np.array(labels)



2024-01-29 21:50:02.701820: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 21:50:02.868802: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 21:50:02.868902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 21:50:02.894653: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 21:50:02.955830: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 21:50:02.957225: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
good_tyre_images, good_tyre_labels = load_and_preprocess_images(good_tyre_path, label=0)
bad_tyre_images, bad_tyre_labels = load_and_preprocess_images(bad_tyre_path, label=1)

# Combine and shuffle the data
all_images = np.concatenate((good_tyre_images, bad_tyre_images), axis=0)
all_labels = np.concatenate((good_tyre_labels, bad_tyre_labels), axis=0)
shuffle_indices = np.random.permutation(len(all_images))
all_images = all_images[shuffle_indices]
all_labels = all_labels[shuffle_indices]

split_ratio = 0.8
split_index = int(len(all_images) * split_ratio)

train_data, train_labels = all_images[:split_index], all_labels[:split_index]
val_data, val_labels = all_images[split_index:], all_labels[split_index:]

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use a learning rate scheduler
def lr_schedule(epoch):
    lr = 0.0001
    if epoch > 60:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)



In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(datagen.flow(train_data, train_labels, batch_size=32), epochs=120,  
          validation_data=(val_data, val_labels), callbacks=[lr_scheduler])

# Save the model
model.save('dude.h5')


Epoch 1/120
47/47 [==============================] - 49s 1s/step - loss: 2.2446 - accuracy: 0.5923 - val_loss: 2.3401 - val_accuracy: 0.4409 - lr: 1.0000e-04
Epoch 2/120
47/47 [==============================] - 48s 1s/step - loss: 1.3630 - accuracy: 0.6759 - val_loss: 0.9384 - val_accuracy: 0.6586 - lr: 1.0000e-04
Epoch 3/120
47/47 [==============================] - 48s 1s/step - loss: 0.8752 - accuracy: 0.6887 - val_loss: 0.6194 - val_accuracy: 0.7419 - lr: 1.0000e-04
Epoch 4/120
47/47 [==============================] - 49s 1s/step - loss: 0.7114 - accuracy: 0.6968 - val_loss: 0.6250 - val_accuracy: 0.7070 - lr: 1.0000e-04
Epoch 5/120
47/47 [==============================] - 48s 1s/step - loss: 0.6412 - accuracy: 0.7210 - val_loss: 0.5412 - val_accuracy: 0.7339 - lr: 1.0000e-04
Epoch 6/120
47/47 [==============================] - 48s 1s/step - loss: 0.5994 - accuracy: 0.7170 - val_loss: 0.4348 - val_accuracy: 0.7930 - lr: 1.0000e-04
Epoch 7/120
47/47 [==============================] -